In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler


In [39]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [40]:
train = train.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test = test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [52]:
train.Embarked = train.Embarked.str.replace('S', '3')
train.Embarked = train.Embarked.str.replace('Q', '2')
train.Embarked = train.Embarked.str.replace('C', '1')
train.Sex = train.Sex.str.replace('male', '1')
train.Sex = train.Sex.str.replace('fe1', '0')

test.Embarked = test.Embarked.str.replace('S', '3')
test.Embarked = test.Embarked.str.replace('Q', '2')
test.Embarked = test.Embarked.str.replace('C', '1')
test.Sex = test.Sex.str.replace('male', '1')
test.Sex = test.Sex.str.replace('fe1', '0')
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,7.8292,2
1,893,3,0,47.0,1,0,7.0000,3
2,894,2,1,62.0,0,0,9.6875,2
3,895,3,1,27.0,0,0,8.6625,3
4,896,3,0,22.0,1,1,12.2875,3


In [53]:
X = train.copy()
y = X.Survived
X = X.drop('Survived', axis=1)

In [54]:
steps = [('imputation', Imputer(missing_values='NaN', strategy='mean',
        axis=0)),
        ('scaler', StandardScaler()),
        ('SVM', SVC())]
pipeline = Pipeline(steps)
parameters = {'SVM__C': [1, 3, 10, 30, 100],
             'SVM__gamma': [0.01, 0.03, 0.1, 0.3, 1]}
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3,
                                                   random_state=21)
cv = GridSearchCV(pipeline, parameters, cv=5)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print("Accuracy: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.8395522388059702
             precision    recall  f1-score   support

          0       0.84      0.90      0.87       161
          1       0.83      0.75      0.79       107

avg / total       0.84      0.84      0.84       268

Tuned Model Parameters: {'SVM__C': 10, 'SVM__gamma': 0.03}


In [55]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,7.8292,2
1,893,3,0,47.0,1,0,7.0000,3
2,894,2,1,62.0,0,0,9.6875,2
3,895,3,1,27.0,0,0,8.6625,3
4,896,3,0,22.0,1,1,12.2875,3


In [56]:
predictions = cv.predict(test)

In [58]:
print(predictions)

[0 1 0 0 1 0 1 0 1 0 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 1 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0
 1 1 1 1 1 1 0 1 0 0 0]


In [59]:
output = pd.DataFrame()
output['PassengerId'] = test.PassengerId
output['Survived'] = predictions
output.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [60]:
output.to_csv('titanic_SVM_1.csv')